In [1]:
!pip install -q GitPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


In [2]:
from git import Repo

repo_url = "https://github.com/Sourajit-Maity/juvdv2-vdvwc"
repo_dir = "juvdv2-vdvwc"  # Replace with your desired directory name

Repo.clone_from(repo_url, repo_dir)

<git.repo.base.Repo '/content/juvdv2-vdvwc/.git'>

In [3]:
!pwd

/content


In [4]:
%cd {repo_dir}

/content/juvdv2-vdvwc


In [5]:
!pwd

/content/juvdv2-vdvwc


In [6]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [7]:
import os
import shutil
import yaml
from sklearn.model_selection import train_test_split
import glob
data = {
    'images': [
        "/content/juvdv2-vdvwc/Train/Rainny/Day",
        "/content/juvdv2-vdvwc/Train/Rainny/Night",
        "/content/juvdv2-vdvwc/Train/Sunny/Day",
        "/content/juvdv2-vdvwc/Train/Sunny/Night",
        "/content/juvdv2-vdvwc/Val/Rainny/Day",
        "/content/juvdv2-vdvwc/Val/Rainny/Night",
        "/content/juvdv2-vdvwc/Val/Sunny/Day",
        "/content/juvdv2-vdvwc/Val/Sunny/Night"
    ],
    'labels': [
        "/content/juvdv2-vdvwc/Annotation/Train/Rainny/Day",
        "/content/juvdv2-vdvwc/Annotation/Train/Rainny/Night",
        "/content/juvdv2-vdvwc/Annotation/Train/Sunny/Day",
        "/content/juvdv2-vdvwc/Annotation/Train/Sunny/Night",
        "/content/juvdv2-vdvwc/Annotation/Val/Rainny/Day",
        "/content/juvdv2-vdvwc/Annotation/Val/Rainny/Night",
        "/content/juvdv2-vdvwc/Annotation/Val/Sunny/Day",
        "/content/juvdv2-vdvwc/Annotation/Val/Sunny/Night"
    ],
    'nc': 14,
    'names': ['car', 'bike', 'auto', 'rickshaw', 'cycle', 'bus', 'minitruck', 'truck', 'van', 'taxi', 'motorvan', 'toto', 'train', 'boat']
}

# Function to extract files from source directory to destination
def extract_files(source_dirs, destination_dir):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    for source_dir in source_dirs:
        for root, _, files in os.walk(source_dir):
            for file in files:
                src_file = os.path.join(root, file)
                dst_file = os.path.join(destination_dir, file)
                shutil.copyfile(src_file, dst_file)

consolidated_images_dir = "/content/juvdv2-vdvwc/datasets/images"
consolidated_labels_dir = "/content/juvdv2-vdvwc/datasets/labels"
extract_files(data['images'], consolidated_images_dir)
extract_files(data['labels'], consolidated_labels_dir)

# Get lists of image and label files
image_files = sorted([f for f in glob.glob(os.path.join(consolidated_images_dir, '*')) if os.path.isfile(f)])
label_files = sorted([f for f in glob.glob(os.path.join(consolidated_labels_dir, '*')) if os.path.isfile(f)])

# Match images and labels by filenames
image_files_base = {os.path.splitext(os.path.basename(f))[0]: f for f in image_files}
label_files_base = {os.path.splitext(os.path.basename(f))[0]: f for f in label_files}

common_keys = set(image_files_base.keys()).intersection(label_files_base.keys())

matched_image_files = [image_files_base[k] for k in common_keys]
matched_label_files = [label_files_base[k] for k in common_keys]

# Split the dataset into train+val and test sets (90% train+val, 10% test)
train_val_images, test_images, train_val_labels, test_labels = train_test_split(
    matched_image_files, matched_label_files, test_size=0.1, random_state=42
)

# Further split train+val into train and val sets (80% train, 20% val)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_val_images, train_val_labels, test_size=0.2, random_state=42
)

# Function to copy files to the destination directory
def copy_files(file_list, destination_dir):
    os.makedirs(destination_dir, exist_ok=True)
    for file in file_list:
        shutil.copy(file, destination_dir)

# Define directories for train, val, and test sets
train_images_dir = "/content/juvdv2-vdvwc/datasets/train/images"
val_images_dir = "/content/juvdv2-vdvwc/datasets/val/images"
test_images_dir = "/content/juvdv2-vdvwc/datasets/test/images"
train_labels_dir = "/content/juvdv2-vdvwc/datasets/train/labels"
val_labels_dir = "/content/juvdv2-vdvwc/datasets/val/labels"
test_labels_dir = "/content/juvdv2-vdvwc/datasets/test/labels"

# Copy the files to the respective directories
copy_files(train_images, train_images_dir)
copy_files(val_images, val_images_dir)
copy_files(test_images, test_images_dir)
copy_files(train_labels, train_labels_dir)
copy_files(val_labels, val_labels_dir)
copy_files(test_labels, test_labels_dir)

print("Dataset split into train, val, and test sets completed!")
print(f"Total number of train images: {len(train_images)}")
print(f"Total number of val images: {len(val_images)}")
print(f"Total number of test images: {len(test_images)}")

dataset_yaml = {
    'train': '/content/juvdv2-vdvwc/datasets/train/images',
    'val': '/content/juvdv2-vdvwc/datasets/val/images',
    'test': '/content/juvdv2-vdvwc/datasets/test/images',
    'nc': 14,
    'names': ['car', 'bike', 'auto', 'rickshaw', 'cycle', 'bus', 'minitruck', 'truck', 'van', 'taxi', 'motorvan', 'toto', 'train', 'boat']
}

dataset_yaml_path = "/content/juvdv2-vdvwc/dataset.yaml"
with open(dataset_yaml_path, 'w') as yaml_file:
    yaml.dump(dataset_yaml, yaml_file, default_flow_style=False)

print(f"dataset.yaml created successfully at {dataset_yaml_path}!")

Dataset split into train, val, and test sets completed!
Total number of train images: 2016
Total number of val images: 504
Total number of test images: 280
dataset.yaml created successfully at /content/juvdv2-vdvwc/dataset.yaml!


In [8]:
import os
import numpy as np

def clean_labels(label_dir, nc):
    """Remove labels with class indices that are out of bounds."""
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        labels = np.loadtxt(label_path)
        if labels.size == 0:
            continue
        if labels.ndim == 1:
            labels = labels.reshape(1, -1)
        valid_labels = labels[labels[:, 0] < nc]
        if valid_labels.shape[0] < labels.shape[0]:
            print(f"Removed invalid labels from {label_file}")
        np.savetxt(label_path, valid_labels, fmt='%f')

train_labels_dir = "/content/juvdv2-vdvwc/datasets/train/labels"
val_labels_dir = "/content/juvdv2-vdvwc/datasets/val/labels"
test_labels_dir = "/content/juvdv2-vdvwc/datasets/test/labels"

nc = 14
clean_labels(train_labels_dir, nc)
clean_labels(val_labels_dir, nc)
clean_labels(test_labels_dir, nc)

print("Label cleaning completed!")

Removed invalid labels from night (34).txt
Removed invalid labels from night (13).txt
Removed invalid labels from night (29).txt
Removed invalid labels from night (42).txt
Removed invalid labels from night (43).txt
Removed invalid labels from night (32).txt
Removed invalid labels from night (33).txt
Removed invalid labels from night (15).txt
Removed invalid labels from night (8).txt
Removed invalid labels from night (45).txt


<ipython-input-8-bf59ba2e4321>:8: UserWarning: loadtxt: input contained no data: "/content/juvdv2-vdvwc/datasets/train/labels/night (79).txt"
  labels = np.loadtxt(label_path)


Removed invalid labels from night (1).txt
Removed invalid labels from night (36).txt
Removed invalid labels from night (28).txt
Removed invalid labels from night (24).txt
Removed invalid labels from night (6).txt
Removed invalid labels from night (44).txt
Removed invalid labels from night (12).txt
Removed invalid labels from night (14).txt
Removed invalid labels from night (49).txt
Removed invalid labels from night (9).txt
Removed invalid labels from night (18).txt
Removed invalid labels from night (38).txt
Removed invalid labels from night (41).txt
Removed invalid labels from night (19).txt
Removed invalid labels from night (27).txt
Removed invalid labels from night (26).txt
Removed invalid labels from night (40).txt
Removed invalid labels from night (25).txt
Removed invalid labels from night (3).txt
Removed invalid labels from night (46).txt
Removed invalid labels from night (48).txt
Removed invalid labels from night (35).txt


<ipython-input-8-bf59ba2e4321>:8: UserWarning: loadtxt: input contained no data: "/content/juvdv2-vdvwc/datasets/train/labels/Sunny-Day (417).txt"
  labels = np.loadtxt(label_path)


Removed invalid labels from night (22).txt
Removed invalid labels from night (10).txt
Removed invalid labels from night (21).txt
Removed invalid labels from night (20).txt
Removed invalid labels from night (31).txt
Removed invalid labels from night (37).txt
Removed invalid labels from night (39).txt
Removed invalid labels from night (2).txt
Removed invalid labels from night (4).txt
Removed invalid labels from night (47).txt
Removed invalid labels from night (30).txt
Removed invalid labels from night (23).txt
Removed invalid labels from night (7).txt
Label cleaning completed!


In [9]:
import torch
from ultralytics import YOLO

def print_batch_info(images, targets):
    """Prints information about the batch of images and targets during training."""
    print(f"Images shape: {images.shape}")
    print(f"Targets shape: {targets.shape}")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device for training.')


model = YOLO("yolov8n.yaml").load("yolov8n.pt")

model.to(device)

print("Starting training...")
model.train(data='dataset.yaml', epochs=25, imgsz=640, batch=8, device=device)

print("Training completed!")

Using cuda device for training.


100%|██████████| 6.25M/6.25M [00:00<00:00, 77.3MB/s]

Transferred 355/355 items from pretrained weights


Starting training...
Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=dataset.yaml, epochs=25, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

100%|██████████| 755k/755k [00:00<00:00, 15.2MB/s]


...

      25/25      1.15G      1.575     0.8964      1.153         48        640: 100%|██████████| 252/252 [00:19<00:00, 13.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:02<00:00, 13.04it/s]

                   all        504       2961      0.842      0.711      0.804      0.428



25 epochs completed in 0.163 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n summary (fused): 168 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:08<00:00,  3.83it/s]


                   all        504       2961      0.828      0.736      0.811      0.436
                   car        395       1695      0.887      0.709      0.815      0.415
                  bike        201        417      0.754      0.618      0.716      0.335
                  auto        113        188       0.86      0.817      0.855      0.432
              rickshaw         61        208      0.881      0.715      0.855      0.352
                 cycle         57         68      0.712      0.583      0.658       0.35
                   bus         63         81      0.862      0.728      0.779      0.381
             minitruck         65         70       0.68      0.671      0.707       0.36
                 truck         57         73      0.821      0.877      0.906      0.494
                   van         55         66      0.737      0.591       0.66      0.367
                  taxi         49         57      0.799      0.579      0.685      0.377
              motorva